# Crude Oil vs Henry Hub Natural Gas Time Series Comparison

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.api import VAR

In [ ]:
file_path = "/mnt/data/CrudeOil_NatGas_Prices.xlsx"
df = pd.read_excel(file_path)

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

df = df.rename(columns={
    df.columns[0]: 'Oil',
    df.columns[1]: 'Gas'
})

df = df.dropna()
df.head()

In [ ]:
plt.figure()
plt.plot(df.index, df['Oil'], label='Crude Oil')
plt.plot(df.index, df['Gas'], label='Henry Hub Gas')
plt.title("Price Levels (USD per MMBtu)")
plt.legend()
plt.show()

In [ ]:
indexed = df / df.iloc[0] * 100

plt.figure()
plt.plot(indexed.index, indexed['Oil'], label='Crude Oil (Indexed)')
plt.plot(indexed.index, indexed['Gas'], label='Henry Hub Gas (Indexed)')
plt.title("Indexed Price Comparison (Base = 100)")
plt.legend()
plt.show()

In [ ]:
returns = np.log(df).diff().dropna()

plt.figure()
plt.plot(returns.index, returns['Oil'], label='Oil Returns')
plt.plot(returns.index, returns['Gas'], label='Gas Returns')
plt.title("Log Returns")
plt.legend()
plt.show()

In [ ]:
returns.describe()

In [ ]:
def adf_test(series, name):
    result = adfuller(series)
    print(f"ADF Test for {name}")
    print(f"Test Statistic: {result[0]:.4f}")
    print(f"P-value: {result[1]:.4f}")
    print("-" * 40)

adf_test(df['Oil'], "Oil Price")
adf_test(df['Gas'], "Gas Price")
adf_test(returns['Oil'], "Oil Returns")
adf_test(returns['Gas'], "Gas Returns")

In [ ]:
returns['Oil'].corr(returns['Gas'])

In [ ]:
rolling_corr = returns['Oil'].rolling(window=60).corr(returns['Gas'])

plt.figure()
plt.plot(rolling_corr)
plt.title("60-Period Rolling Correlation (Returns)")
plt.axhline(0)
plt.show()

In [ ]:
rolling_vol = returns.rolling(window=60).std()

plt.figure()
plt.plot(rolling_vol.index, rolling_vol['Oil'], label='Oil Volatility')
plt.plot(rolling_vol.index, rolling_vol['Gas'], label='Gas Volatility')
plt.title("Rolling Volatility (60 Periods)")
plt.legend()
plt.show()

In [ ]:
coint_stat, p_value, crit_vals = coint(df['Oil'], df['Gas'])
coint_stat, p_value

In [ ]:
model = VAR(returns)
lag_order = model.select_order(maxlags=10)
lag_order.summary()

In [ ]:
var_model = model.fit(lag_order.selected_orders['aic'])
var_model.summary()

In [ ]:
irf = var_model.irf(10)
irf.plot()
plt.show()